In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cv2
from PIL import Image
import os
from matplotlib import pyplot as plt
from copy import deepcopy
import pickle
import seaborn as sns
import pandas as pd
from masking import centre_mask, non_centre_mask, random_mask
import time

#Turn all the randomisation off to ensure the results of every execution is the same 
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [2]:
IMG_SIZE = 224
FOLDER_PATH = "Images/100"

masks = {
    "Centre": centre_mask,
    "Non-centre": non_centre_mask,
    "Random": random_mask 
}

classification_models = ["CNN", "KNN", "SVM", "Random Forest"]

models_masks_accuracies = {model: {mask: [] for mask in masks.keys()}
                               for model in classification_models}
models_masks_f1_scores = {model: {mask: [] for mask in masks.keys()}
                               for model in classification_models}
models_masks_classification_times = {model: {mask: [] for mask in masks.keys()}
                               for model in classification_models}

def build_transform(mask: str) -> transforms.Compose:
    mask = masks.get(mask)

    return transforms.Compose([
        transforms.Lambda(mask),
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [ ]:
class ImageDataSet(Dataset):
    def __init__(self, image_names, transform):
        self.file_names = []
        self.labels = []
        for numeric_label, names in enumerate(image_names):
            self.labels.extend([numeric_label]*len(names))
            self.file_names.extend(names)

        self.transform = transform

    def __getitem__(self, index):
        img_name = self.file_names[index]
        img = Image.open(img_name).convert('RGB')
        img = self.transform(img)
        label = self.labels[index]
        return img, label
    
    def __len__(self):
        return len(self.file_names)
    

labels = ["Immune_Cells", "Non_Invasive_Tumor", "Invasive_Tumor_Set"]
le = LabelEncoder()
numeric_labels = le.fit_transform(labels)
image_names = []
for _ in numeric_labels:
    image_names.append([])

for (dir_path, dir_names, file_names) in os.walk(FOLDER_PATH):
    parent_folder = os.path.basename(dir_path)
    if parent_folder in labels: # Read the subset of dataset to reduce training time 
        for file in file_names:
            image = cv2.imread(os.path.join(dir_path, file))
            if image.shape[0] < 100 and image.shape[1] < 100: #skip the small image, it doesn't give much info
                continue
            numeric_label = le.transform([parent_folder])[0]
            image_names[numeric_label].append(os.path.join(dir_path, file))



masking_datasets = {key : ImageDataSet(image_names, build_transform(key)) for key in masks.keys()}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #check if the computer has GPU

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(image_names))
model = model.to(device)

masking_cnn_models = {key : deepcopy(model).to(device) for key in masks.keys()}

In [ ]:
# hyper-parameters setting
num_epochs = 100
patience = 10 #for early stopping
batch_size = 128
learning_rate = 0.001

In [ ]:
def feature_extractor(model, train_loader, test_loader):
    model.eval()
    feature_extractor = nn.Sequential(*list(model.children())[:-1]) # remove the last layer
    feature_extractor.eval()
    feature_extractor.to(device)

    train_features = []
    train_labels = []
    test_features = []
    test_labels = []

    with torch.no_grad():
        for images, labels in train_loader:
            images = images.to(device)
            output = feature_extractor(images).squeeze()
            train_features.append(output.cpu().numpy())
            train_labels.append(labels.cpu().numpy())

    X_train = np.vstack(train_features)
    y_train = np.hstack(train_labels)

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            output = feature_extractor(images).squeeze()
            test_features.append(output.cpu().numpy())
            test_labels.append(labels.cpu().numpy())

    X_test = np.vstack(test_features)
    y_test = np.hstack(test_labels)

    return X_train, y_train, X_test, y_test

In [ ]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

for mask, model in masking_cnn_models.items():
    print(f"Start training {mask} model.")
    dataset = masking_datasets[mask]
    all_indices, _ = train_test_split(list(range(len(dataset))), test_size=0.5, random_state=0) #only use 50% of the dataset
    all_labels = [dataset[i][1] for i in all_indices]

    for train_idx, test_idx in kf.split(all_indices, all_labels):
        train_idx, val_idx = train_test_split(train_idx, test_size=0.2, random_state=0)

        train_isx_set = set(train_idx)
        val_isx_set = set(val_idx)
        test_isx_set = set(test_idx)
        print(train_isx_set.intersection(val_isx_set))
        print(train_isx_set.intersection(test_isx_set))
        print(val_isx_set.intersection(test_isx_set))
        print("-------------")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        test_subset = Subset(dataset, test_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=False, pin_memory=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, pin_memory=True)
        test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False, pin_memory=True)

        best_val_loss = float('inf')
        epoch_no_improvement = 0
        best_model_parameters = None
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            avg_val_loss = val_loss / len(val_loader)
            val_accuracy = 100 * correct / total
            print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                epoch_no_improvement = 0
                best_model_parameters = deepcopy(model.state_dict())
            else:
                epoch_no_improvement += 1
                if epoch_no_improvement >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    break


        if best_model_parameters is not None:
            model.load_state_dict(best_model_parameters)      
        
        model.eval()
        y_true = []
        y_pred = []

        start = time.time()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())
        end = time.time()

        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        elapsed_time = end - start

        models_masks_accuracies["CNN"][mask].append(accuracy)
        models_masks_f1_scores["CNN"][mask].append(f1)
        models_masks_classification_times["CNN"][mask].append(elapsed_time)

        print(f"CNN - Test Accuracy: {accuracy:.2f}%, Test F1 Score: {f1:.4f}, Classification Time: {elapsed_time:.2f} seconds")

        feature_extraction_start = time.time()
        X_train, y_train, X_test, y_test = feature_extractor(model, train_loader, test_loader)
        feature_extraction_end = time.time()
        feature_extraction_time = feature_extraction_end - feature_extraction_start

        svm = SVC()
        svm.fit(X_train, y_train)
        start = time.time()
        y_pred = svm.predict(X_test)
        end = time.time()
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        classification_time = end - start + feature_extraction_time

        models_masks_accuracies["SVM"][mask].append(accuracy)
        models_masks_f1_scores["SVM"][mask].append(f1)
        models_masks_classification_times["SVM"][mask].append(classification_time)
        print(f"SVM - Test Accuracy: {accuracy:.2f}%, Test F1 Score: {f1:.4f}, Classification Time: {classification_time:.2f} seconds")

        rf = RandomForestClassifier()
        rf.fit(X_train, y_train)
        start = time.time()
        y_pred = rf.predict(X_test)
        end = time.time()
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        classification_time = end - start + feature_extraction_time

        models_masks_accuracies["Random Forest"][mask].append(accuracy)
        models_masks_f1_scores["Random Forest"][mask].append(f1)
        models_masks_classification_times["Random Forest"][mask].append(classification_time)
        print(f"Random Forest - Test Accuracy: {accuracy:.2f}%, Test F1 Score: {f1:.4f}, Classification Time: {classification_time:.2f} seconds")

        knn_accuracies = []
        knn_f1_scores = []
        knn_classification_times = []
        for k in range(1, 32, 2): #k = 1 to 31
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)

            start = time.time()
            y_pred = knn.predict(X_test)
            end = time.time()
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            classification_time = end - start + feature_extraction_time

            knn_accuracies.append(accuracy)
            knn_f1_scores.append(f1)
            knn_classification_times.append(classification_time)

        knn_accuracies = np.array(knn_accuracies)
        max_idx = np.argmax(knn_accuracies)
        best_k = 2*max_idx+1

        accuracy = knn_accuracies[max_idx]
        f1 = knn_f1_scores[max_idx]
        classification_time = knn_classification_times[max_idx]

        models_masks_accuracies["KNN"][mask].append(accuracy)
        models_masks_f1_scores["KNN"][mask].append(f1)
        models_masks_classification_times["KNN"][mask].append(classification_time)
        print(f"KNN - Test Accuracy: {accuracy:.2f}%, Test F1 Score: {f1:.4f}, Classification Time: {classification_time:.2f} seconds")

In [ ]:
with open("masking_cv_results/models_masks_accuracies.pkl", "wb") as f:
    pickle.dump(models_masks_accuracies, f)
with open("masking_cv_results/models_masks_f1_scores.pkl", "wb") as f:
    pickle.dump(models_masks_f1_scores, f)
with open("masking_cv_results/models_masks_classification_times.pkl", "wb") as f:
    pickle.dump(models_masks_classification_times, f)

In [ ]:
df_nested = pd.DataFrame(models_masks_accuracies).T  # Models become rows
df_nested.reset_index(inplace=True)
df_nested = df_nested.melt(id_vars='index', var_name='Mask', value_name='Accuracy')
df_nested.columns = ['Model', 'Mask', 'Accuracy(%)']

df_long = df_nested.explode('Accuracy(%)', ignore_index=True)

plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=df_long, x='Model', y='Accuracy(%)', hue='Mask')

# Add vertical lines between model groups
xticks = ax.get_xticks()
for i in range(1, len(xticks)):
    plt.axvline(x=(xticks[i-1] + xticks[i]) / 2, color='gray', linestyle='--', linewidth=1)

plt.title("Accuracy by Model and Mask Method")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
df_nested = pd.DataFrame(models_masks_f1_scores).T  # Models become rows
df_nested.reset_index(inplace=True)
df_nested = df_nested.melt(id_vars='index', var_name='Mask', value_name='F1_Score')
df_nested.columns = ['Model', 'Mask', 'F1 Score']

df_long = df_nested.explode('F1 Score', ignore_index=True)

plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=df_long, x='Model', y='F1 Score', hue='Mask')

# Add vertical lines between model groups
xticks = ax.get_xticks()
for i in range(1, len(xticks)):
    plt.axvline(x=(xticks[i-1] + xticks[i]) / 2, color='gray', linestyle='--', linewidth=1)

plt.title("F1 Score by Model and Mask Method")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
df_nested = pd.DataFrame(models_masks_classification_times).T  # Models become rows
df_nested.reset_index(inplace=True)
df_nested = df_nested.melt(id_vars='index', var_name='Mask', value_name='Time(s)')
df_nested.columns = ['Model', 'Mask', 'Time(s)']

df_long = df_nested.explode('Time(s)', ignore_index=True)

plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=df_long, x='Model', y='Time(s)', hue='Mask')

# Add vertical lines between model groups
xticks = ax.get_xticks()
for i in range(1, len(xticks)):
    plt.axvline(x=(xticks[i-1] + xticks[i]) / 2, color='gray', linestyle='--', linewidth=1)

plt.title("Classification Time by Model and Mask Method")
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()